# Install dependencies

In [1]:
%pip install --user labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations split-folders imutils scikit-learn keras 

  Using cached labelme-5.1.1-py3-none-any.whl
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached albumentations-1.3.0-py3-none-any.whl (123 kB)
  Using cached split_folders-0.5.1-py3-none-any.whl (8.4 kB)
  Using cached scikit_learn-1.2.1-cp310-cp310-win_amd64.whl (8.3 MB)
  Using cached natsort-8.2.0-py3-none-any.whl (37 kB)
  Using cached PyYAML-6.0-cp310-cp310-win_amd64.whl (151 kB)
  Using cached imgviz-1.7.2-py3-none-any.whl (7.7 MB)
  Using cached QtPy-2.3.0-py3-none-any.whl (83 kB)
  Using cached opencv_python_headless-4.7.0.68-cp37-abi3-win_amd64.whl (38.1 MB)
  Using cached qudida-0.0.4-py3-none-any.whl (3.5 kB)
  Using cached scikit_image-0.19.3-cp310-cp310-win_amd64.whl (12.0 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached imageio-2.25.1-py3-none-any.whl (3.4 MB)
  Using cached PyWavelets-1.4.1-cp310-cp310-win_amd64.whl (4.2 MB)
  Us

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Harry Parker\AppData\Local\Temp\pip-install-1yf2hi16\tensorflow-gpu_3d08c99bd9e64ee580d4c0b9951b7451\setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subpr

## Import libraries

In [27]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import csv
import re
from PIL import Image

In [28]:

img = plt.imread("./data/WIDERFace/WIDER_train/images/0--Parade/0_Parade_Parade_0_904.jpg")
img = cv2.rectangle(img,(361,98),(263+361,339+98),color=(255,0,0),thickness=3)
plt.figure(figsize=(15,10))
plt.imshow(img)

ModuleNotFoundError: No module named 'matplotlib.backends'

In [5]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

tf.config.list_physical_devices('GPU')

[]

# Load dataset

In [6]:
baseMapNum = 32
weight_decay = 1e-4
# ANCHORS = [0.57273, 0.677385]
ANCHORS = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]
BOX = 5
GRID_H = GRID_W = 13
BATCH_SIZE = 4
IMAGE_H, IMAGE_W = 440,440
OBJ_THRESHOLD    = 0.5
NMS_THRESHOLD    = 0.5
NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
WARM_UP_BATCHES  = 0
TRUE_BOX_BUFFER  = 50

In [42]:

"""Utility functions for creating TFRecord data sets."""

import tensorflow as tf


def int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def int64_list_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def bytes_list_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def float_list_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def read_examples_list(path):
  """Read list of training or validation examples.
  The file is assumed to contain a single example per line where the first
  token in the line is an identifier that allows us to find the image and
  annotation xml for that example.
  For example, the line:
  xyz 3
  would allow us to find files xyz.jpg and xyz.xml (the 3 would be ignored).
  Args:
    path: absolute path to examples list file.
  Returns:
    list of example identifiers (strings).
  """
  with tf.gfile.GFile(path) as fid:
    lines = fid.readlines()
  return [line.strip().split(' ')[0] for line in lines]


def recursive_parse_xml_to_dict(xml):
  """Recursively parses XML contents to python dict.
  We assume that `object` tags are the only ones that can appear
  multiple times at the same level of a tree.
  Args:
    xml: xml tree obtained by parsing XML file contents using lxml.etree
  Returns:
    Python dictionary holding XML contents.
  """
  if not xml:
    return {xml.tag: xml.text}
  result = {}
  for child in xml:
    child_result = recursive_parse_xml_to_dict(child)
    if child.tag != 'object':
      result[child.tag] = child_result[child.tag]
    else:
      if child.tag not in result:
        result[child.tag] = []
      result[child.tag].append(child_result[child.tag])
  return {xml.tag: result}

In [44]:
# Training
TRAIN_WIDER_PATH = "./data/WIDERFace/WIDER_train/"

# Validation
VAL_WIDER_PATH = "./data/WIDERFace/WIDER_val/"

# Testing
TEST_WIDER_PATH = "./data/WIDERFace/WIDER_test/"

# Ground Truth
GROUND_TRUTH_PATH = "./data/WIDERFace/wider_face_split/"

# Output
OUTPUT_PATH = ".data/WIDERFace/output/"

import tensorflow as tf
import numpy
import cv2
import os
import hashlib


def parse_test_example(f, images_path):
    height = None # Image height
    width = None # Image width
    filename = None # Filename of the image. Empty if image is not from file
    encoded_image_data = None # Encoded image bytes
    image_format = b'jpeg' # b'jpeg' or b'png'

    filename = f.readline().rstrip()
    if not filename:
        raise IOError()

    filepath = os.path.join(images_path, filename)

    image_raw = cv2.imread(filepath)

    encoded_image_data = open(filepath, "rb").read()
    key = hashlib.sha256(encoded_image_data).hexdigest()

    height, width, channel = image_raw.shape

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(int(height)),
        'image/width': int64_feature(int(width)),
        'image/filename': bytes_feature(filename.encode('utf-8')),
        'image/source_id': bytes_feature(filename.encode('utf-8')),
        'image/key/sha256': bytes_feature(key.encode('utf8')),
        'image/encoded': bytes_feature(encoded_image_data),
        'image/format': bytes_feature('jpeg'.encode('utf8')),
        }))


    return tf_example


def parse_example(f, images_path):
    height = None # Image height
    width = None # Image width
    filename = None # Filename of the image. Empty if image is not from file
    encoded_image_data = None # Encoded image bytes
    image_format = b'jpeg' # b'jpeg' or b'png'

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)
    poses = []
    truncated = []
    difficult_obj = []

    filename = f.readline().rstrip()
    if not filename:
        raise IOError()

    filepath = os.path.join(images_path, filename)

    image_raw = cv2.imread(filepath)

    encoded_image_data = open(filepath, "rb").read()
    key = hashlib.sha256(encoded_image_data).hexdigest()

    height, width, channel = image_raw.shape

    face_num = int(f.readline().rstrip())
    if not face_num:
        raise Exception()

    for i in range(face_num):
        annot = f.readline().rstrip().split()
        if not annot:
            raise Exception()

        # WIDER FACE DATASET CONTAINS SOME ANNOTATIONS WHAT EXCEEDS THE IMAGE BOUNDARY
        if(float(annot[2]) > 25.0):
            if(float(annot[3]) > 30.0):
                xmins.append( max(0.005, (float(annot[0]) / width) ) )
                ymins.append( max(0.005, (float(annot[1]) / height) ) )
                xmaxs.append( min(0.995, ((float(annot[0]) + float(annot[2])) / width) ) )
                ymaxs.append( min(0.995, ((float(annot[1]) + float(annot[3])) / height) ) )
                classes_text.append(b'face')
                classes.append(1)
                poses.append("front".encode('utf8'))
                truncated.append(int(0))


    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(int(height)),
        'image/width': int64_feature(int(width)),
        'image/filename': bytes_feature(filename.encode('utf-8')),
        'image/source_id': bytes_feature(filename.encode('utf-8')),
        'image/key/sha256': bytes_feature(key.encode('utf8')),
        'image/encoded': bytes_feature(encoded_image_data),
        'image/format': bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': float_list_feature(xmins),
        'image/object/bbox/xmax': float_list_feature(xmaxs),
        'image/object/bbox/ymin': float_list_feature(ymins),
        'image/object/bbox/ymax': float_list_feature(ymaxs),
        'image/object/class/text': bytes_list_feature(classes_text),
        'image/object/class/label': int64_list_feature(classes),
        'image/object/difficult': int64_list_feature(int(0)),
        'image/object/truncated': int64_list_feature(truncated),
        'image/object/view': bytes_list_feature(poses),
        }))


    return tf_example


def run(images_path, description_file, output_path, no_bbox=False):
    f = open(description_file)
    writer = tf.python_io.TFRecordWriter(output_path)

    i = 0

    print("Processing {}".format(images_path))
    while True:
        try:
            if no_bbox:
                tf_example = parse_test_example(f, images_path)
            else:
                tf_example = parse_example(f, images_path)

            writer.write(tf_example.SerializeToString())
            i += 1

        except IOError:
            break
        except Exception:
            raise

    writer.close()

    print("Correctly created record for {} images\n".format(i))


def main(unused_argv):
    # Training
    if TRAIN_WIDER_PATH is not None:
        images_path = os.path.join(TRAIN_WIDER_PATH, "images")
        description_file = os.path.join(GROUND_TRUTH_PATH, "wider_face_train_bbx_gt.txt")
        output_path = os.path.join(OUTPUT_PATH, "train.tfrecord")
        run(images_path, description_file, output_path)

    # Validation
    if VAL_WIDER_PATH is not None:
        images_path = os.path.join(VAL_WIDER_PATH, "images")
        description_file = os.path.join(GROUND_TRUTH_PATH, "wider_face_val_bbx_gt.txt")
        output_path = os.path.join(OUTPUT_PATH, "val.tfrecord")
        run(images_path, description_file, output_path)

    # Testing. This set does not contain bounding boxes, so the tfrecord will contain images only
    if TEST_WIDER_PATH is not None:
        images_path = os.path.join(TEST_WIDER_PATH, "images")
        description_file = os.path.join(GROUND_TRUTH_PATH, "wider_face_test_filelist.txt")
        output_path = os.path.join(OUTPUT_PATH, "test.tfrecord")
        run(images_path, description_file, output_path, no_bbox=True)

# View images

In [ ]:
image_generator = images.batch(4).as_numpy_iterator()

plot_images = image_generator.next()

fig, axes = plt.subplots(ncols=4, figsize=(20, 20))
for i, image in enumerate(plot_images):
    axes[i].imshow(image)
plt.show()




# Combine images and labels

In [ ]:
len(train_images), len(test_images), len(val_images), len(train_labels), len(val_labels)

train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(5000)
train = train.batch(64)
train = train.prefetch(8)

##test = tf.data.Dataset.zip((test_images, test_labels))
#test = test.shuffle(1500)
#test = test.batch(64)
#test = test.prefetch(8)

val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1000)
val = val.batch(64)
val = val.prefetch(8)

train.as_numpy_iterator().next()[1]

data_samples = train.as_numpy_iterator().next()

res = data_samples.next()

fig, axes = plt.subplots(ncols=4, figsize=(20, 20))
for i in range(4):
    samples_image = res[0][i]
    sample_coords = res[1][1][i]

    cv2.rectangle(samples_image, 
                  tuple(np.multiply(sample_coords[:2], [120, 120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:], [120, 120]).astype(int)),
                  (255, 0, 0), 2)

axes[i].imshow(samples_image)

    

# Functional API

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from keras.applications import VGG16

vgg = VGG16(include_top=False)

vgg.summary()

In [14]:
#building classifer 
def build_model():
    model = Sequential()
    model.add(Conv2D(8*baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),
                     input_shape=(IMAGE_H, IMAGE_W, 3),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(Conv2D(8*baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(4 * baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(Conv2D(4 * baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))  

    model.add(Conv2D(2 * baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(Conv2D(2 * baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))  
    
    model.add(Conv2D(2 * baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.1))  
   
    model.add(Conv2D(baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(Conv2D(baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation=LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())

    model.add(Conv2D(BOX * (4 + 1), (1,1), strides=(1,1)))
    model.add(Reshape((GRID_H, GRID_W,BOX, 4 + 1)))

    return model

faceDetectionModel = build_model()
faceDetectionModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 440, 440, 256)     7168      
                                                                 
 batch_normalization (BatchN  (None, 440, 440, 256)    1024      
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 440, 440, 256)     590080    
                                                                 
 batch_normalization_1 (Batc  (None, 440, 440, 256)    1024      
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 220, 220, 256)    0         
 )                                                               
                                                        

In [ ]:
X, y = train.as_numpy_iterator().next()
X.shape

classes, coords = faceDetectionModel.predict(X)

classes, coords


# Losses

In [19]:
faceDetectionModel.compile(loss=custom_loss, optimizer=tf.keras.optimizers.Adam())

In [20]:
def custom_loss(y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]
    print("Mask",np.shape(mask_shape))

    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(GRID_W), [GRID_H]), (1, GRID_H, GRID_W, 1, 1)))
    print("cell_x",np.shape(cell_x))
    cell_y = cell_x
    print("cell_y",np.shape(cell_y))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [BATCH_SIZE, 1, 1, 5, 1])
    print("cell_grid",np.shape(cell_grid))

    coord_mask = tf.zeros(mask_shape)
    conf_mask  = tf.zeros(mask_shape)
    print("coord_mask",np.shape(coord_mask))
    
    seen = tf.Variable(0.)
    total_recall = tf.Variable(0.)
    
    """
    Adjust prediction
    """
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(ANCHORS, [1,1,1,BOX,2])    
    pred_box_conf = tf.sigmoid(y_pred[..., 4])
    """
    Adjust ground truth
    """
    true_box_xy = y_true[..., 0:2] 
    true_box_wh = y_true[..., 2:4] 
    true_box_conf = y_true[..., 4]
    """
    Determine the masks
    """
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 4], axis=-1) * COORD_SCALE
    """
    Warm-up training
    """
    no_boxes_mask = tf.to_float(coord_mask < COORD_SCALE/2.)
    
    seen = tf.assign_add(seen, 1.)
    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, WARM_UP_BATCHES),
                   lambda: [true_box_xy + (0.5) * no_boxes_mask, 
                   true_box_wh + tf.ones_like(true_box_wh) * np.reshape(ANCHORS, [1,1,1,BOX,2])* no_boxes_mask, tf.ones_like(coord_mask)],
                   lambda: [true_box_xy, true_box_wh, coord_mask])
    
 
    """
    Finalize the loss
    """
    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    # nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))

    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(tf.sqrt(true_box_wh)-tf.sqrt(pred_box_wh))     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf))
    loss = loss_xy + loss_wh + loss_conf
    
    return loss

# Old losses

In [ ]:
batches_per_epoch = len(train_images)
lr_decay = (1./0.75-1)/batches_per_epoch

optimiser = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)

def localisation_loss(y_true, yhat):
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2]))

    h_true = y_true[:,3] - y_true[:,1]
    w_true = y_true[:,2] - y_true[:,0]

    h_pred = yhat[:,3] - yhat[:,1]
    w_pred = yhat[:,2] - yhat[:,0]

    delta_size = tf.reduce_sum(tf.square(tf.sqrth_true - w_true) + tf.square(h_pred - w_pred))

    return delta_coord + delta_size

classloss = tf.keras.losses.BinaryCrossentropy()
regloss = localisation_loss

localisation_loss(y[1], coords)

classloss(y[0], classes)

regloss(y[1], coords)

# Train CNN

In [ ]:
class FaceDetection(Model):
      def __init__(self, facecctv, **kwargs):
        super().__init__(**kwargs)
        self.model = facecctv

      def compile(self, optimiser, classloss, localisationloss, **kwargs):
          super().compile(**kwargs)
          self.classloss = classloss
          self.localisationloss = localisationloss
          self.optimiser = optimiser
      
      def train_step(self, batch, **kwargs):
          
            X, y = batch

            with tf.GradientTape() as tape:
                  classes, coords = self.model(X, training=True)

                  batch_class_loss = self.classloss(y[0], classes)

                  batch_localisation_loss = self.localisationloss(tf.cast(y[1], tf.float32), coords)

                  total_loss = 0.5*batch_class_loss + batch_localisation_loss

                  gradients = tape.gradient(total_loss, self.model.trainable_variables)

            optimiser.apply_gradients(zip(gradients, self.model.trainable_variables))

            return {"class_loss": batch_class_loss, "localisation_loss": batch_localisation_loss, "total_loss": total_loss}
      
      def test_step(self, batch, **kwargs):
            X, y = batch

            classes, coords = self.model(X, training=False)

            batch_class_loss = self.classloss(y[0], classes)

            batch_localisation_loss = self.localisationloss(tf.cast(y[1], tf.float32), coords)

            total_loss = 0.5*batch_class_loss + batch_localisation_loss

            return {"class_loss": batch_class_loss, "localisation_loss": batch_localisation_loss, "total_loss": total_loss}
          
      def call(self, X, **kwargs):
            return self.model(X, **kwargs)
      
model = FaceDetection(faceDetectionModel)
model.compile(optimiser, classloss, regloss)

logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir, histogram_freq = 1)
hist = model.fit(train, epochs=10, validation_data=val, callbacks=[tensorboard_callback])

hist.history

fig, axes = plt.subplots(ncols=3, figsize=(20, 5))

axes[0].plot(hist.history['total_loss'], color='teal', label='total_loss')
axes[0].plot(hist.history['val_total_loss'], color='orange', label='val_total_loss')
axes[0].set_title('Total Loss')
axes[0].legend()

axes[1].plot(hist.history['class_loss'], color='teal', label='class_loss')
axes[1].plot(hist.history['val_class_loss'], color='orange', label='val_class_loss')
axes[1].set_title('Classification Loss')
axes[1].legend()

axes[2].plot(hist.history['localisation_loss'], color='teal', label='localisation_loss')
axes[2].plot(hist.history['val_localisation_loss'], color='orange', label='val_localisation_loss')
axes[2].set_title('Localisation Loss')
axes[2].legend()

plt.show()

# Train model

In [22]:
i=0
i, x, y = GenerateBatch(i,500)
print(i)
x_train, x_test, y_train ,y_test = train_test_split(x, y, test_size=0.2, random_state=50)
print(np.shape(y_train))
y_train = np.reshape(y_train,(len(x_train),13, 13, BOX, 5))
print(np.shape(y_train))
     

NameError: name 'x' is not defined

In [23]:
faceDetectionModel.fit(x_train, y_train, epochs=50, batch_size=BATCH_SIZE, validation_data=(x_test,y_test), verbose=1)
faceDetectionModel.save_weights("faceDetectionModel.h5")

NameError: name 'x_train' is not defined

# Test model

In [ ]:
print(np.exp(0.5))
print(np.log(np.exp(0.05)))
     

import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

_, tempx, ty = GenerateBatch(0,1000)
img = tempx[0]
true_box_wh = ty[..., 2:4]
print(np.shape(true_box_wh))
i = ty[0].reshape((GRID_H * GRID_W,5))[67]
print(i)
Nx,Ny, Nw, Nh = ReAdjustCord(i[0]*(8/GRID_W),i[1]*(5/GRID_H), i[2], i[3],img)
cv2.rectangle(img, (int(Nx),int(Ny)), (int(Nw)+int(Nx),int(Nh)+int(Ny)), (255,0,0)) 
cv2.imshow(img)
print(Nx,Ny, Nw, Nh)
tempy = model.predict(tempx)
i = tempy[0].reshape((GRID_H * GRID_W,5))[67]
Nx,Ny, Nw, Nh = sigmoid(i[0]),sigmoid(i[1]), np.exp(i[2]), np.exp(i[3])
print(Nx,Ny, Nw, Nh)
Nx,Ny, Nw, Nh = ReAdjustCord(Nx*(8/GRID_W),Ny*(5/GRID_H), Nw, Nh,img)
print(Nx,Ny, Nw, Nh)
cv2.rectangle(img, (int(Nx),int(Ny)), (int(Nw)+int(Nx),int(Nh)+int(Ny)), (255,0,0)) 
cv2.imshow(img)

In [ ]:
for i in range(GRID_H * GRID_W):
  tt = ty[0].reshape((GRID_H * GRID_W,5))[i]
  if(tt[4] > 0.5):
    print(tt)
    print(i)

In [ ]:
_, tempx, ty = GenerateBatch(0,1000)
tempy = model.predict(tempx)
img = tempx[0]
print(np.shape(tempy[0]))
for idx,i in enumerate(ty[0].reshape((9,5))):
    if(float(i[4]) > 0.5):
      print("ssss",i,idx)
      Nx,Ny, Nw, Nh = sigmoid(i[0]),sigmoid(i[1]), i[2], i[3]
      print(Nx,Ny, Nw, Nh)
      Nx,Ny, Nw, Nh = ReAdjustCord(i[0],i[1], i[2], i[3],img)
      print(Nx,Ny, Nw, Nh)
      cv2.rectangle(img, (int(Nx),int(Ny)), (int(Nw)+int(Nx),int(Nh)+int(Ny)), (255,0,0)) 
      cv2.imshow(img)

In [ ]:
model.save_weights("weights.h5")

In [ ]:
predicted = model.predict(x)
predicted = np.reshape(predicted,(1,3*3,5))
for i in predicted[0]:
      if(float(i[0]) > 0.5):
            print(i)
print(np.sum(predicted,axis = 1))
print(np.sum(np.reshape(y,(1,3*3,5)),axis = 1))

# Predictions 

In [ ]:
test_data = test.as_nupmy_iterator().next()
test_samples = test_data.next()
yhat = faceDetectionModel.predict(test_samples[0])

fig, axes = plt.subplots(ncols=4, figsize=(20, 20))
for i in range(4):
      sample_image = test_samples[0][i]
      sample_coords = yhat[1][i]

      if yhat[0][i] > 0.9:
            cv2.rectangle(sample_image,
                        tuple(np.multiply(sample_coords[:2], 120).astype(int)),
                        tuple(np.multiply(sample_coords[2:], 120).astype(int)),
                        (255, 0, 0), 2)
            
      axes[i].imshow(sample_image)

# Save model

In [ ]:
from keras.models import load_model

faceDetectionModel.save('faceDetectionModel.h5')
faceDetectionModel = load_model('faceDetectionModel.h5', custom_objects={'localisation_loss': localisation_loss})

faceDetectionModel.summary()

def detect_face(image, model):
      image = cv2.resize(image, (120, 120))
      image = image.astype('float32')/255.
      image = np.expand_dims(image, axis=0)
      classes, coords = model.predict(image)
      if classes[0][0] > 0.9:
            return coords[0]
      else:
            return None
